In [2]:
#### CODE ADOPTED FROM https://towardsdatascience.com/lstm-vs-bert-a-step-by-step-guide-for-tweet-sentiment-analysis-ced697948c47 ####

In [9]:
import numpy as np
import torch

from pandas import read_csv
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset# create Tensor datasets

In [14]:
# LOAD FILE DAN BUAT LIST
filepath = 'dataset.csv'
column_names = ["Tweet", "Label"]

df = read_csv(filepath, header=0, sep=',', names=column_names)
messages = df.Tweet.to_list()
sentiments_ = df.Label.to_list()

sentiments = [1 if sentiment =='Setuju Vaksin' else 0 for sentiment in sentiments_]
sentiments = np.array(sentiments_)

#print(df)
#print('----------------------------------------------------------------')
#print(messages)
#print('----------------------------------------------------------------')
print(sentiments[0:10])

['Setuju Vaksin' 'Setuju Vaksin' 'Setuju Vaksin' 'Setuju Vaksin'
 'Setuju Vaksin' 'Setuju Vaksin' 'Setuju Vaksin' 'Setuju Vaksin'
 'Setuju Vaksin' 'Setuju Vaksin']
